
# Deep Learning

## Assignment 2

Previously in 1_notmnist.ipynb, we created a pickle with formatted datasets for training, development and testing on the notMNIST dataset.

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

首先加载第一部分处理的数据
+ data as a flat matrix
+ labels as float 1-hot encodings.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print('Training set', train_dataset.shape, train_labels.shape)
    print('Validation set', valid_dataset.shape, valid_labels.shape)
    print('Test set', test_dataset.shape, test_labels.shape)


Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


重新格式化数据的形状，便于更好的训练模型

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels


train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)


Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [9]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random values following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf_test_dataset, weights) + biases)

In [21]:
num_steps = 801

def accuracy(predictions, labels):
    return (100*np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])


with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases.
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' %(step, l))
            print('Training accuracy: %.1f%%' % accuracy(
                predictions, train_labels[:train_subset, :]))
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(
        test_prediction.eval(), test_labels))


Initialized
Loss at step 0: 17.618397
Training accuracy: 8.4%
Validation accuracy: 10.9%
Loss at step 100: 2.273187
Training accuracy: 71.8%
Validation accuracy: 71.3%
Loss at step 200: 1.835675
Training accuracy: 75.0%
Validation accuracy: 73.6%
Loss at step 300: 1.604544
Training accuracy: 76.2%
Validation accuracy: 74.1%
Loss at step 400: 1.447317
Training accuracy: 77.1%
Validation accuracy: 74.6%
Loss at step 500: 1.328135
Training accuracy: 77.8%
Validation accuracy: 74.9%
Loss at step 600: 1.232803
Training accuracy: 78.3%
Validation accuracy: 75.1%
Loss at step 700: 1.154293
Training accuracy: 78.9%
Validation accuracy: 75.4%
Loss at step 800: 1.088357
Training accuracy: 79.5%
Validation accuracy: 75.5%
Test accuracy: 82.8%


### 使用小批量随机梯度下降算法

In [24]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(
        tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    weights = tf.Variable(
        tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(
        tf.matmul(tf_test_dataset, weights) + biases)


In [30]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = { tf_train_dataset: batch_data,
                     tf_train_labels: batch_labels }
        _, l, predictions = session.run(
            [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" %
                  accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
                valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(
        test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 15.309130
Minibatch accuracy: 12.5%
Validation accuracy: 17.7%
Minibatch loss at step 500: 1.962580
Minibatch accuracy: 71.9%
Validation accuracy: 75.2%
Minibatch loss at step 1000: 0.865288
Minibatch accuracy: 82.8%
Validation accuracy: 76.7%
Minibatch loss at step 1500: 1.328339
Minibatch accuracy: 77.3%
Validation accuracy: 77.4%
Minibatch loss at step 2000: 1.108631
Minibatch accuracy: 71.9%
Validation accuracy: 78.1%
Minibatch loss at step 2500: 1.045632
Minibatch accuracy: 74.2%
Validation accuracy: 77.8%
Minibatch loss at step 3000: 0.801308
Minibatch accuracy: 85.9%
Validation accuracy: 78.8%
Test accuracy: 86.2%


### 加入hidden layer

In [50]:
batch_size = 128
hidden_layer_size = 1024

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                      shape=(None, image_size * image_size))
    tf_train_labels = tf.placeholder(
        tf.float32, shape=(None, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # hidden layer
    weights_hidden = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    biases_hidden = tf.Variable(tf.zeros([hidden_layer_size]))
    hidden = tf.nn.relu(tf.matmul(tf_train_dataset, weights_hidden) + biases_hidden)

    # output.
    weights = tf.Variable(
        tf.truncated_normal([hidden_layer_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    logits = tf.matmul(hidden, weights) + biases
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
     # 模型评估
    correct_prediction = tf.equal(tf.argmax(tf_train_labels,1), tf.argmax(logits,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    

In [61]:
num_steps = 4001

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    validate_feed = { tf_train_dataset : valid_dataset, \
                     tf_train_labels : valid_labels}
    test_feed = { tf_train_dataset : test_dataset, \
                     tf_train_labels : test_labels}
    tf.global_variables_initializer().run()
    print("Initialized")
    
    for step in range(num_steps):
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        train_feed = { tf_train_dataset: batch_data,
                     tf_train_labels: batch_labels }
        _, l, train_accuracy = sess.run(
            [optimizer, loss, accuracy], feed_dict=train_feed)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f" %train_accuracy)
            valid_accuracy = sess.run(accuracy, feed_dict=validate_feed)
            print("Validation accuracy: %f" % valid_accuracy)
    test_accuracy = sess.run(accuracy, feed_dict=test_feed)
    print("Test accuracy: %f" %test_accuracy)

Initialized
Minibatch loss at step 0: 370.830048
Minibatch accuracy: 0.1
Validation accuracy: 0.301400
Minibatch loss at step 500: 8.785034
Minibatch accuracy: 0.8
Validation accuracy: 0.809300
Minibatch loss at step 1000: 9.752355
Minibatch accuracy: 0.8
Validation accuracy: 0.798900
Minibatch loss at step 1500: 18.154409
Minibatch accuracy: 0.8
Validation accuracy: 0.801600
Minibatch loss at step 2000: 5.312876
Minibatch accuracy: 0.8
Validation accuracy: 0.809400
Minibatch loss at step 2500: 3.607777
Minibatch accuracy: 0.8
Validation accuracy: 0.820600
Minibatch loss at step 3000: 6.647066
Minibatch accuracy: 0.9
Validation accuracy: 0.790700
Minibatch loss at step 3500: 1.946131
Minibatch accuracy: 0.8
Validation accuracy: 0.807700
Minibatch loss at step 4000: 6.632562
Minibatch accuracy: 0.8
Validation accuracy: 0.815200
Test accuracy: 0.884400
